In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os
import time
import pickle as pkl
from tqdm import tqdm

from t2_functions import partition_ids

In [87]:
import sys

sys.path.append(os.path.join(os.getcwd(), "task_2_sybilattack/"))

from endpoints.requests import sybil, sybil_reset


In [89]:
print(os.getcwd())
dataset = torch.load("data/SybilAttack.pt")

/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack


In [90]:
ids = np.array(dataset.ids)
binned_ids = partition_ids(ids, main_bin_num=10)

bin_size: 2000


In [57]:
def is_linearly_independent(set_of_vectors, vector):
    # Check if the new vector is linearly independent of the set
    coefficients = np.linalg.lstsq(set_of_vectors, vector, rcond=None)[0]
    return np.linalg.norm(vector - np.dot(set_of_vectors, coefficients)) > 1e-10

def find_independent_set(vectors, k=200):
    vectors = np.array(vectors)
    n = len(vectors)
    independent_set = vectors[:k]
    indexes = np.arange(k)
    i = 1
    for j, vector in enumerate(tqdm(vectors)):
        if i >= k:
            return np.array(independent_set), indexes, i
        if n - j < k - i:
            return np.array(independent_set), indexes, i
        if is_linearly_independent(np.array(independent_set[:i]).T, vector):
            independent_set[i] = vector
            indexes[i] = j
            i += 1
    return np.array(independent_set), indexes, i


In [82]:
task = 'binary'

sybil_reset(home_or_defense='home', binary_or_affine=task)
sybil_reset(home_or_defense='defense', binary_or_affine=task)


Request ok
{'msg': 'Successful sybil binary reset home'}
Request ok
{'msg': 'Successful sybil binary reset defense'}


In [83]:
ids = binned_ids[0]
print(len(ids))
A_reps = sybil(ids=ids,
               home_or_defense='home',
               binary_or_affine=task)

A_indep, indexes, success = find_independent_set(A_reps, k=200)
print(f"success: {success}")
print(indexes)

ids_train = ids[indexes]
mask = np.ones(len(ids), dtype=bool)
mask[indexes] = False
ids_test = ids[mask]

A_reps = np.array(A_reps)
A_train_reps = A_reps[indexes]
A_test_reps = A_reps[mask]

B_train_reps = sybil(ids=ids_train,
                 home_or_defense='defense',
                 binary_or_affine=task)
print(f"B train reps: {len(B_train_reps)}")

B_test_reps = sybil(ids=ids_test,
                 home_or_defense='defense',
                 binary_or_affine=task)
print(f"B test reps: {len(B_test_reps)}")


2000


 10%|█         | 200/2000 [00:00<00:04, 400.62it/s]


success: 200
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199]
A train reps: 2000
A test reps: 1800


In [84]:
task='affine_v3'

os.makedirs(f'data/{task}', exist_ok=True)

with open(f'data/{task}/A_train', 'wb') as f:
    pkl.dump(A_train_reps, f)
with open(f'data/{task}/B_train', 'wb') as f:
    pkl.dump(B_train_reps, f)
with open(f'data/{task}/A_test', 'wb') as f:
    pkl.dump(A_test_reps, f)
with open(f'data/{task}/B_test', 'wb') as f:
    pkl.dump(B_test_reps, f)


In [85]:
with open(f'data/{task}/A_train', 'rb') as f:
    A_train_reps = pkl.load(f)
with open(f'data/{task}/B_train', 'rb') as f:
    B_train_reps = pkl.load(f)
with open(f'data/{task}/A_test', 'rb') as f:
    A_test_reps = pkl.load(f)
with open(f'data/{task}/B_test', 'rb') as f:
    B_test_reps = pkl.load(f)
    

In [29]:
class RepresentationsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]
        return torch.tensor(x), torch.tensor(y)


In [30]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [31]:
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super(Linear, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [32]:
def validate(criterion, loader, net):
    net.eval()

    with torch.no_grad():
        true = []
        pred = []
        for x, y in loader:
            y_hat = net(x)
            true.append(y.numpy())
            pred.append(y_hat.numpy())

        true = torch.tensor(np.concatenate(true, axis=0))
        pred = torch.tensor(np.concatenate(pred, axis=0))

        loss = criterion(pred, true)

    return loss


In [33]:
def l1_reg(model, reg_lambda):
    l1_regularization = torch.tensor(0., device=model.parameters().__next__().device)
    for param in model.parameters():
        l1_regularization += torch.norm(param, p=1)
    return reg_lambda * l1_regularization


In [34]:
from torch.utils.tensorboard import SummaryWriter

def train(epochs, optim, criterion, regularise, trainloader, valloader, net, empty_net, reg_lambda=None):
    best_val_loss = np.inf
    best_net = empty_net
    writer = SummaryWriter()


    for epoch in range(epochs):  # loop over the dataset multiple times
        train_loss = 0

        progress_bar = tqdm(trainloader)

        for iter, (x, y) in enumerate(progress_bar):
            net.train()
            optim.zero_grad()

            y_hat = net(x)
            loss = criterion(y_hat, y)
            if reg_lambda is not None:
                loss += regularise(net, reg_lambda)
            loss.backward()
            optim.step()

            batch_loss = loss.item()
            train_loss += batch_loss
            if iter % 20 == 0:
                progress_bar.set_description(f"train | loss: {batch_loss:.4f}")

            writer.add_scalar('Training Loss', batch_loss, epoch)

        train_loss /= len(trainloader)
        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}")

        net.eval()

        val_loss = validate(criterion, valloader, net)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_net.load_state_dict(net.state_dict())

        if writer is not None:
            # log the validation loss and accuracy
            writer.add_scalar('Validation Loss', val_loss, epoch)

        print(f"Epoch [{epoch + 1}/{epochs}], Val Loss: {val_loss:.4f}")

    return net, best_net



In [35]:
criterion = nn.MSELoss()
batch_size = 32
epochs = 100
lr = 0.001


In [36]:
train_dataset = RepresentationsDataset(x=A_train_reps, y=B_train_reps)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = RepresentationsDataset(x=A_test_reps, y=B_test_reps)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [37]:
lin_net = Linear(384, 384)
lin_empty_net = Linear(384, 384)

optim = torch.optim.Adam(lr=lr, params=lin_net.parameters())

lin_last_net, lin_best_net = train(epochs, optim, criterion, l1_reg, train_loader, test_loader, lin_net, lin_empty_net, reg_lambda=0.0001)

train | loss: 4.1753: 100%|██████████| 7/7 [00:00<00:00, 57.14it/s]


Epoch [1/100], Train Loss: 2.5125
Epoch [1/100], Val Loss: 1.1748


train | loss: 1.6496: 100%|██████████| 7/7 [00:00<00:00, 34.01it/s]


Epoch [2/100], Train Loss: 1.3977
Epoch [2/100], Val Loss: 0.8715


train | loss: 1.2516: 100%|██████████| 7/7 [00:00<00:00, 98.36it/s]

...

train | loss: 0.1265: 100%|██████████| 7/7 [00:00<00:00, 25.56it/s]


Epoch [99/100], Train Loss: 0.1259
Epoch [99/100], Val Loss: 0.1150


train | loss: 0.1329: 100%|██████████| 7/7 [00:00<00:00, 46.02it/s]

Epoch [100/100], Train Loss: 0.1249


Epoch [100/100], Val Loss: 0.1144


In [ ]:
mlp = MLP(384, 384, 384)
empty_mpl = MLP(384, 384, 384)

optim = torch.optim.Adam(lr=lr, params=mlp.parameters())

mlp_last_net, mlp_best_net = train(epochs, optim, criterion, l1_reg, train_loader, test_loader, mlp, empty_mpl, reg_lambda=0.001)

In [39]:
test_loss_lin = validate(criterion, test_loader, lin_last_net)
print(f"lin test: {test_loss_lin}")

train_loss_lin = validate(criterion, train_loader, lin_last_net)
print(f"lin train: {train_loss_lin}")


lin test: 0.11437146365642548
lin train: 0.06165158003568649
